Aim
=====

The aim of this session is to familiarize yourselves with decision trees, as well as their bagged and boosted versions, which are famous ML models for precision medicine problems due to their high interpretability. For the purposes of this session, we are going to use a publicly available dataset, the heart failure prediction dataset that can be found [here](zhttps://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction).

In the last part of this session, we will explore ways to improve your overall classification pipeline that are not specific to decision trees and ensembles based on trees but are important to know.

Please note that we will not perform exploratory data analysis as you have already done this for this dataset in a previous session.

Dataset:
========
The dataset contains data from 918 individuals and was created with the aim of identifying people who have cardiovascular disease (CVD) or are at risk of developing CVD. In total, 11 features are available:

1. Age: age of the patient [years]
2. Sex: sex of the patient [M: Male, F: Female]
3. ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, 4. NAP: Non-Anginal Pain, ASY: Asymptomatic]
4. RestingBP: resting blood pressure [mm Hg]
5. Cholesterol: serum cholesterol [mm/dl]
6. FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
7. RestingECG: resting electrocardiogram results – measures the electrical activity of the heart [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
8. MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]
9. ExerciseAngina: exercise-induced angina [Y: Yes, N: No]
10. Oldpeak: ST [Numeric value measured in depression]
11. ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
Additionally the column HeartDisease provides the output class [1: heart disease, 0: Normal]

The dataset was adopted from: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

Author: Polyxeni Gkontra, Machine Learning for Precision Medicine, MBDS, 2024

Data Preparation
============

Import the required libraries

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import io
# Libraries related to classification
from sklearn.metrics import classification_report, roc_curve, ConfusionMatrixDisplay, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler

Load the file with the data to colab

In [ ]:
from google.colab import files
# Load the file - a window will prompt to choose from your local system
uploaded = files.upload()

Read the .csv file with the patient information. Then create a dataframe X that contains only the features and another one that contains the labels. Useful: [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from pandas

In [ ]:
# Read the csv file
data = pd.read_csv(io.BytesIO(uploaded['hd_data.csv']))

In [ ]:
# Column name that contains the class information
label = 'HeartDisease'
# Dataframe with the labels
Y =pd.DataFrame(columns=[label])
Y[label] = data[label].copy()
# Drop the column with the classes from the X data so as to create a dataframe containing only features
X = data.drop(label, axis=1)

Print the number of patients in each class and create a relevant plot. Useful: [countplot](https://seaborn.pydata.org/generated/seaborn.countplot.html)

In [ ]:
# The number of individuals in each class

# Create a countplot with the number of patients per category


Check for missing values (Helpful: Function [isnull](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) from [pandas](https://pandas.pydata.org/docs/))

Basic statistics (Helpful: Methods [describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), [info](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) from pandas)

Split the dataset into training (80%) and testing (20%). We will leave the testing set aside and further split the training into training and validation. Useful: [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
X_train, X_test, Y_train, Y_test =

Check first rows of X_train

In [ ]:
X_train.head()

Transform categorical variables using one hot-key encoding for sex and ordinal encoding for the rest

In [ ]:
# Check the type of the features
X_train.dtypes

Get the categorical features and their indexes as well as the index of the sex attribute to treat it differently during encoding.

In [ ]:
# Numerical features
num_features = list(X_train.select_dtypes(include=["int64", "float64"]).columns)
# Indices of numerical features
numerical_idx = [loc for loc, key in enumerate(X_train.columns) if key in num_features]
# Indices of columns with categorical data
categorical_idx = list(set(range(0,X_train.shape[1])) - set(numerical_idx))
# Index of sex attribute
sex_idx = [loc for loc, key in enumerate(X_train.columns) if key == 'Sex']
# Delete sex from the indices of categorical features
categorical_idx.remove(sex_idx[0])
# Names of categorical features
cat_features = X_train.columns[categorical_idx]
# Print all features
print("All available features: ", X_train.columns)
# Print numerical features
print("Numerical features: ", num_features)
# Print categorical features
print("Categorical features: ", cat_features)

Encode the sex attribute by means of one-hot key encoding and the rest using ordinal encoding. Useful: [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn-preprocessing-ordinalencoder)

In [ ]:
# Function to help with one hot key encoding with pandas. Reproduced from
# https://stackoverflow.com/questions/37292872/how-can-i-one-hot-encode-in-python
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

In [ ]:
# importing copy module
import copy
# Create copies of the features
X_train_enc = copy.deepcopy(X_train);
X_test_enc = copy.deepcopy(X_test);

# Get one hot key encoding for sex attribute
X_train_enc = encode_and_bind(X_train_enc, 'Sex')
X_test_enc = encode_and_bind(X_test_enc, 'Sex')

# Handle all other categorical variables using ordinal encoder


# Apply normalization to numerical variables
#num_preprocesssor = MinMaxScaler()
#X_train_enc[num_features] = num_preprocesssor.fit_transform(X_train_enc[num_features])
#X_test_enc[num_features] = num_preprocesssor.transform(X_test_enc[num_features])

In [ ]:
# Name of the columns
X_train_enc.columns

Transfrom the labels using the [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [ ]:
# Define transform for the target labels - attention! LabelEncoder is for target variables only!

# Transform the training data
Y_train_enc =
# Transform the testing data
Y_test_enc =

Decision trees
=====
Decision trees are considered ¨white box¨ models allowing to completely understand how a decision was made by the algorithm. Here, we are going to use then to predict whether a patient has or is at risk of heart disease.

Train a decision tree model and apply it to the testing data. Evaluate the performance of the model in terms of precision, recall, f1 score and roc auc. Moreover, calculate and visualize the confusion matrix. Useful: [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn-tree-decisiontreeclassifier), [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html), [ConfusionMatrixDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html)

In [ ]:
from sklearn import tree

In [ ]:
# Define the decision tree model to be used
tree_model = tree.DecisionTreeClassifier(criterion="entropy", max_depth=2)

# Fit the model to the training data


# Apply the trained model to the testing data
Y_tree1 =

# Evaluate the performance of the model


# Calculate confusion matrix
cm_tree1 = confusion_matrix(Y_test_enc, Y_tree1)
# Plot the confusion matrix
display = ConfusionMatrixDisplay(confusion_matrix=cm_tree1, display_labels = ['Healthy', 'Diseased'])
fig, ax = plt.subplots(figsize=(5,5))
display.plot(ax=ax, xticks_rotation='vertical')
plt.show()

Let's now make a change. Let's change the balanced weight and other parameters

Import the [graphviz](https://graphviz.readthedocs.io/en/stable/index.html) library that allows us to visualize graphs

In [ ]:
import graphviz

Export the tree in DOT format. You can akso save it in a file with filename tree1.pdf. Useful: [export_graphviz](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html#sklearn-tree-export-graphviz)

In [ ]:
dot_data = tree.export_graphviz(tree_model, out_file=None,
                     feature_names=X_train_enc.columns,
                     class_names=['Healthy', 'Diseased'],
                     filled=True, rounded=True,
                   special_characters=True)
graph = graphviz.Source(dot_data)
# Render the graph
graph
# Save the graph
#graph.render("tree1")


 You can also automatically download the created file

In [ ]:
# You can also download the file for better visualization
from google.colab import files
files.download("tree1.pdf")

Get the most important features. Add them in an array in descending order of importance and create a relevant plot with the feature importance. Useful: feature_importances_, feature_names_in_ (check out [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) attributes for more details), [argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html)

In [ ]:
# Sort the feature in descending order of importance
sorted_idx = tree_model.feature_importances_.argsort()
# Create the plot
plt.barh(tree_model.feature_names_in_[sorted_idx], tree_model.feature_importances_[sorted_idx])
plt.xlabel("Feature Importance for model's decision");

Random forests
======================

Random forests are popular enseble examples of decision trees based on bagging. Let's see how they work in our problem. Useful: [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Define the random forest model to be used


# Fit the model to the training data


# Apply the trained model to the testing data to make predictions on the unseen data


# Evaluate the performance of the model


Create the plot with the most important features for the classifier's decision

In [ ]:
# Sort the feature in descending order of importance

# Create the plot


Visualize one of the trees of the model, eg. the first one. Useful: estimators_ attribute from the [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn-ensemble-randomforestclassifier) object

In [ ]:

# Render the graph

# Save the graph
# graph.render("RF")

Experiment with different number of trees, depths and criteria for splitting. Do your results change?

XGBoost
========


XGBoost is a powerful ensemble algorithm based on decision trees and boosting.

Import the necessary modules from the [xgboost library](https://xgboost.readthedocs.io/en/stable/).

In [ ]:
from xgboost import XGBClassifier, plot_tree, to_graphviz

Train a XGBoost model and test it. Useful: XGBClassifier. XGBClassifier
has a lot of parameters but the most important are considered:
(the text below is reproduced from https://gist.github.com/pb111/cc341409081dffa5e9eaf60d79562a03):-

"1. learning_rate - It gives us the step size shrinkage which is used to prevent overfitting. Its range is [0,1].
2. max_depth - It determines how deeply each tree is allowed to grow during any boosting round.
3. subsample - It determines the percentage of samples used per tree. Low value of subsample can lead to underfitting.
4. colsample_bytree - It determines the percentage of features used per tree. High value of it can lead to overfitting.
5. n_estimators - It is the number of trees we want to build.
6. objective - It determines the loss function to be used in the process. For example, reg:linear for regression problems, reg:logistic for classification problems with only decision, binary:logistic for classification problems with probability.

XGBoost also supports regularization parameters to penalize models as they become more complex and reduce them to simple models. These regularization parameters are as follows:-
1. gamma - It controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits. It is supported only for tree-based learners.
2. alpha - It gives us the L1 regularization on leaf weights. A large value of it leads to more regularization.
3. lambda - It gives us the L2 regularization on leaf weights and is smoother than L1 regularization."

In [ ]:
# Initialize the model
xgb_model = XGBClassifier(n_estimators = 10, max_depth = 6)

# Fit the model to the training data

# Apply the trained model to the testing data


# Evaluate the performance of the model


# Create confusion matrix


In [ ]:
# Sort the feature in descending order of importance

# Create the plot


In [ ]:
!pip install graphviz

In [ ]:
format = 'png'
graph = to_graphviz(xgb_model, num_trees=0)
graph
# graph.graph_attr = {'dpi':'400'}
#graph.render('filename', format = format)

Check whether preprocessing has any impact

Steps to further improve your classification pipeline: Handling of imbalancing, Preprocessing & Automated hyper-parameter tuning
=====================


In this section, we will discuss some ways to improve your classification pipeline:

1. First, we will address imbalanced data. Class imbalance, where the number of samples from one class is higher than that of the other, is a common issue in the medical domain. The class with the higher amount of data is commonly referred to as the majority class, while the class with the smaller amount of data is called the minority class. Imbalanced data require special measures to be taken; otherwise, we will end up with a model that does not generalize well. To address this, we will utilize pipeline from the imbalanced-learn library.

2. Second, we will perform additional preprocessing steps, including scaling and encoding. We will use pipeline and ColumnTransformer to apply different techniques to different columns, such as numerical data and categorical data, including the 'sex' attribute.

3. Finally, we will incorporate an option for automated hyperparameter tuning using grid search.

 Useful: [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn-compose-columntransformer)

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Downsampling
sampling = RandomUnderSampler(random_state=0)

# Use column transformer to perform a different pipeline for categorical features, sex and numeric
# Encoding and imputation depending on the type of data, i.e. categorical or numeric
t = [('cat', OrdinalEncoder(), categorical_idx), ('sex', OneHotEncoder(), sex_idx), ('num', MinMaxScaler(), numerical_idx)]
preprocess = ColumnTransformer(transformers=t)

# Define the classifier
rf = RandomForestClassifier(100, random_state=42)

model = Pipeline([
        ('sampling', sampling),
        ('preprocessing', preprocess),
        ('clf', rf)
    ])

#============================== Performing GridSearch ==========================
# Set params to explore
# max_features: features considered for the random subset, min_samples_split: The minimum number of samples required to split an internal node
params = {"clf__n_estimators": [5, 10, 100], "clf__max_depth": [9, 5, 3, 2, None], "clf__min_samples_split": [2, 3, 5, 7, 10], "clf__bootstrap": [True]}
grid = GridSearchCV(model, params, scoring='balanced_accuracy')
grid.fit(X_train, Y_train_enc)
#===============================================================================

# Train the model without using gridsearch
# model.fit(X_train, Y_train_enc)

# Apply the trained model to the testing dat
Y_rf_grid = grid.predict(X_test)

# Evaluate the performance of the model
print(classification_report(Y_test_enc, Y_rf_grid))
# Balanced accuracy
print("Balanced accuracy", balanced_accuracy_score(Y_test_enc, Y_rf_grid))

Print the grid search results

In [ ]:
print("Grid Search Results:" )
print("Best estimator:\n",grid.best_estimator_)
print("Best score achieved by best estimator:\n",grid.best_score_)
print("Best parameters across the searched parameters:\n",grid.best_params_)